In [ ]:
#  Last amended:30th August, 2021
#  Myfolder: /home/ashok/Documents/spark
# Ref:
# Tutorials (slightly dated):
#      https://changhsinlee.com/pyspark-dataframe-basics/
#      https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/
# Cheat Sheet
#      https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf

#  Objectives:
#           Dataframe operations in spark cluster

pyspark APIs<br>
> i)  [DataFrame APIs](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#dataframe-apis)<br>
>> df.select(columnName).where(colObject > 30).orderBy(desc(columnName))<br>
>> df.select(columnName).where("colName > 30").orderBy(desc(columnName))<br>

> ii) [Column APIs](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#column-apis)<br>
>> df.select(df.age.isNull())<br>
>> df.select(df["age"].isNull())<br>
>> df.select(col("age").isNull())<br>

> iii)[Data Tyoes](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#data-types)<br>
> iv) [Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions)<br>
>> df.select(sum("age"))<br>
>> df.select(sum(col(booleanColumn).cast("int")))<br>
>> <u>but you must import the functions</u>

> v)  [Grouping](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#grouping)<br>

A. Initial operations:
1.0 Start hadoop in a terminal:

            ./allstart.sh

## Transfer files to hadoop

In [ ]:
# 1.1 Transfer data file 'blackfridayless.csv' to hadoop
#     Linux File folder:  /cdata/misc_datasets/black_friday
#     In Hadoop first make a folder: /user/ashok/datadir 
#     and then transfer the file 'blackfridayless.csv' to 
#     this folder: /user/ashok/datadir

"""

cd ~
hdfs dfs -rm -f -r  /user/ashok/datadir
hdfs dfs -mkdir /user/ashok/datadir
hdfs dfs -put /cdata/misc_datasets/black_friday/blackfridayless.csv  /user/ashok/datadir
hdfs dfs -ls /user/ashok/datadir






"""

## Set jupyter notebook options
Start pyspark with jupyter notebook interface. There is no need to create SparkContext and Spark session. pyspark creates them when starting.

In [1]:
# 1.3 Display multiple outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# 1.4 Increase cell width to display wide columnar output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Read the csv file from hadoop

In [3]:
###### Read file 'blackfridayless.csv' from hadoop

# What is the URL of my file?
URL_of_file= "hdfs://localhost:9000/user/ashok/datadir/"

# 5.2 Takes time. We use 'spark' session object to read file:
blackfriday = spark.read.csv(
                             path = URL_of_file + "blackfridayless.csv",
                             inferSchema = True,      # Default: False
                             header = True,           # Default: False
                             sep = ",",               # Default: ","
                             ignoreLeadingWhiteSpace = True,  # Default: False
                             ignoreTrailingWhiteSpace = True  # Default: False
    
                            )

## Explore the dataframe

In [4]:
blackfriday.show(3)
print(blackfriday.columns)
print(blackfriday.dtypes)

+-------+---------+------+----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
| userId|productId|gender| age|occupation|cityCategory|stayCityYears|maritalStatus|productCat1|productCat2|productCat3|purchase|
+-------+---------+------+----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
|1000001|P00069042|     F|0-17|        10|           A|            2|            0|          3|       null|       null|    8370|
|1000001|P00248942|     F|0-17|        10|           A|            2|            0|          1|          6|         14|   15200|
|1000001|P00087842|     F|0-17|        10|           A|            2|            0|         12|       null|       null|    1422|
+-------+---------+------+----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
only showing top 3 rows

['userId', 'productId', 'gender', 'age', 'occupation', 'cityCategory', '

In [24]:
# Print schema of blackfriday:

blackfriday.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- productId: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- occupation: integer (nullable = true)
 |-- cityCategory: string (nullable = true)
 |-- stayCityYears: string (nullable = true)
 |-- maritalStatus: integer (nullable = true)
 |-- productCat1: integer (nullable = true)
 |-- productCat2: integer (nullable = true)
 |-- productCat3: integer (nullable = true)
 |-- purchase: integer (nullable = true)



In [29]:
# Describe the statistics of data:

blackfriday.select(["userId", "productId", "gender", "age"]).describe().show()
blackfriday.select(["occupation", "cityCategory", "stayCityYears", "maritalStatus"]).describe().show()
blackfriday.select(["productCat1", "productCat2", "productCat3", "purchase"]).describe().show()

+-------+------------------+---------+------+------+
|summary|            userId|productId|gender|   age|
+-------+------------------+---------+------+------+
|  count|            100887|   100887|100887|100887|
|   mean|1002746.9106723364|     null|  null|  null|
| stddev|1678.5336050997462|     null|  null|  null|
|    min|           1000001|P00000142|     F|  0-17|
|    max|           1006040| P0099942|     M|   55+|
+-------+------------------+---------+------+------+

+-------+-----------------+------------+------------------+-------------------+
|summary|       occupation|cityCategory|     stayCityYears|      maritalStatus|
+-------+-----------------+------------+------------------+-------------------+
|  count|           100887|      100887|            100887|             100887|
|   mean|8.086631577903992|        null|1.4730108801539887|0.40921030459821384|
| stddev|6.529747259316683|        null|0.9918812951876441|0.49169058110724656|
|    min|                0|           A|  

In [5]:
# Count How many distinct userids are there 
blackfriday.select(["userid"]).distinct().count()
# Count how many distinct age-groups exist
blackfriday.select(["age"]).distinct().count()

5801

7

In [54]:
# How many null values occur in each column
from pyspark.sql.functions import isnan, isnull,col, sum, max

In [9]:
for i in blackfriday.columns:
    blackfriday.select(sum(col(i).isNull().alias("nullcol").cast("int")).alias(i)).show()    

+------+
|userId|
+------+
|     0|
+------+

+---------+
|productId|
+---------+
|        0|
+---------+

+------+
|gender|
+------+
|     0|
+------+

+---+
|age|
+---+
|  0|
+---+

+----------+
|occupation|
+----------+
|         0|
+----------+

+------------+
|cityCategory|
+------------+
|           0|
+------------+

+-------------+
|stayCityYears|
+-------------+
|            0|
+-------------+

+-------------+
|maritalStatus|
+-------------+
|            0|
+-------------+

+-----------+
|productCat1|
+-----------+
|          0|
+-----------+

+-----------+
|productCat2|
+-----------+
|      31429|
+-----------+

+-----------+
|productCat3|
+-----------+
|      70100|
+-----------+

+--------+
|purchase|
+--------+
|       0|
+--------+



These columns have null values. Most probably it means that there is no sub-category or sub-categories present. <br>
How would you plan to fill them?<br>
productCat1 :  0 <br>
productCat2 :  31429 <br>
productCat3 :  70100 <br>

In [10]:
# Get a list of all integer columns and string columns
intCols = [i[0] for i in blackfriday.dtypes  if i[1] == "int" ]
strCols = [i[0] for i in blackfriday.dtypes  if i[1] == "string" ]

In [11]:
print(intCols)
print(strCols)

['userId', 'occupation', 'maritalStatus', 'productCat1', 'productCat2', 'productCat3', 'purchase']
['productId', 'gender', 'age', 'cityCategory', 'stayCityYears']


In [12]:
blackfriday.select(max('productCat2')).show()
blackfriday.select(max('productCat3')).show()

+----------------+
|max(productCat2)|
+----------------+
|              18|
+----------------+

+----------------+
|max(productCat3)|
+----------------+
|              18|
+----------------+



In [13]:
# Find minimum and max values of each column
blackfriday.select(max(col('occupation'))).show()

+---------------+
|max(occupation)|
+---------------+
|             20|
+---------------+



In [14]:
# Fill null values in productCat2 and productCat3

blackfriday.na.fill(999, ["productCat2"]).show()

+-------+---------+------+-----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
| userId|productId|gender|  age|occupation|cityCategory|stayCityYears|maritalStatus|productCat1|productCat2|productCat3|purchase|
+-------+---------+------+-----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
|1000001|P00069042|     F| 0-17|        10|           A|            2|            0|          3|        999|       null|    8370|
|1000001|P00248942|     F| 0-17|        10|           A|            2|            0|          1|          6|         14|   15200|
|1000001|P00087842|     F| 0-17|        10|           A|            2|            0|         12|        999|       null|    1422|
|1000001|P00085442|     F| 0-17|        10|           A|            2|            0|         12|         14|       null|    1057|
|1000002|P00285442|     M|  55+|        16|           C|           4+|            0|      

In [16]:
# Transform spark dataframe to pandas dataframe:

x = blackfriday.toPandas()
x.head()

,userId,productId,gender,age,occupation,cityCategory,stayCityYears,maritalStatus,productCat1,productCat2,productCat3,purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [19]:
# Show a value count of levels of column 'cityCategory':

blackfriday.groupBy("cityCategory").count().show()

3

+------------+-----+
|cityCategory|count|
+------------+-----+
|           B|42347|
|           C|31071|
|           A|27469|
+------------+-----+



In [22]:
# Perform a stratified sampling of data.
# Stratified sampling be by column: 'cityCategory'
# Take 80% from 'B' and 20% from 'C'

sample = blackfriday.sampleBy(
                      "cityCategory",      # column that defines strata
                      fractions = {'B' : 0.8, 'C' : 0.2})   # sampling fraction for each stratum

## Using verbs
select, <i>select(x).where()</i>, <i>select().distinct()</i>, filter, groupby

### select syntax
> DataFrame.select(\*cols)<br>
> cols: column names (string) or expressions (Column). If one of the column names is ‘*’, that column is expanded to include all columns in the current DataFrame.


In [ ]:
# Show columns 3rd till 5th



### filter syntax
>DataFrame.filter(condition)<br>
>condition: <i>columnObject > 34</i> or string format: <i>"age > 34"</i>
>>  df.age > 3 or col("age") > 3<br>
>>  "age > 3" <br>
>>Logical Operators<br>
>>> If string: AND OR NOT<br>
>>> If columnObject: &, |, ~ <br>

In [33]:
# Filter purchases less than 9000

blackfriday.filter("purchase < 9000")
blackfriday.where("purchase < 9000")
blackfriday.filter(col('purchase') < 9000)

DataFrame[userId: int, productId: string, gender: string, age: string, occupation: int, cityCategory: string, stayCityYears: string, maritalStatus: int, productCat1: int, productCat2: int, productCat3: int, purchase: int]

DataFrame[userId: int, productId: string, gender: string, age: string, occupation: int, cityCategory: string, stayCityYears: string, maritalStatus: int, productCat1: int, productCat2: int, productCat3: int, purchase: int]

DataFrame[userId: int, productId: string, gender: string, age: string, occupation: int, cityCategory: string, stayCityYears: string, maritalStatus: int, productCat1: int, productCat2: int, productCat3: int, purchase: int]

In [46]:
# Filter purchases less than 9000 and maritalStatus is 0

blackfriday.filter('purchase < 9000').filter('maritalStatus = 0').count()
blackfriday.filter('purchase < 9000').filter('maritalStatus == 0').count()
blackfriday.filter('purchase < 9000 AND maritalStatus == 0').count()
blackfriday.filter((col('purchase') < 9000) & (col('maritalStatus') == 0)).count()

35936

35936

100887

35936

In [47]:
# Filter purchases less than 9000 or maritalStatus is 0

blackfriday.filter('purchase < 9000 OR maritalStatus == 0').count()
blackfriday.filter((col('purchase') < 9000) | (col('maritalStatus') == 0)).count()

84313

84313

In [ ]:
# 10. Combining verbs: select, filter and distinct

airports_df.select('dst', 'tz'). \
            filter(airports_df.tz == -5). \
            show(3)

# 10.1
airports_df.select('dst', 'tz'). \
            filter(airports_df.tz == -5). \
            distinct(). \
            show(3)

## Aggregation with groupby
Use: <i>.agg({'colName1' : 'mean', 'colName2' : 'sum'})</i> <br>
>With <i>agg()</i> one can use only builtin functions and not any other <i>pyspark.sql.function</i>.<br>
Some common functions are: <i>mean, avg, sum, count, first, last,stddev </i>. There is no need to import builtin function in advance.<br>
For a complete list of builtin functions see [here](https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/).


In [56]:
blackfriday.agg({'maritalStatus' : 'max', 'purchase' : max}).show()

AttributeError: 'function' object has no attribute '_get_object_id'

In [ ]:
# 12. groupby. Can apply sum, min, max, count

airports_df.groupby('tz'). \
           count(). \
           show(3)

# 12.1
airports_df.groupby('tz'). \
            agg({'lat' : 'mean'}). \
            show(3)

In [ ]:
"""
Unpacking operator in python (*) : 
Ref: https://codeyarns.com/2012/04/26/unpack-operator-in-python/ 

def fox(a,b):  
    return (a *b)  

m = [3,4]  
fox(m)    
fox(*m)   

"""

In [ ]:
# 7.1 One can take the average of columns by passing
#       an unpacked list of column names.

grObject = airports_df.groupby('tz')

avg_cols = ['lat', 'lon']
grObject.avg(*avg_cols).show(3)

In [ ]:
# 7.2 To call multiple aggregation functions at once, pass a dictionary.
#         The 'key' of dictionary becomes argument to 'value'.
#                             count(*)        avg(lat)      sum(lon)

grObject.agg({'*': 'count', 'lat': 'avg', 'lon':'sum'}).show(2)

## Column manipulation

In [ ]:
# 8. Create new columns in Spark using .withColumn() --mutate
#      New column: altInThousands . 
#      Product of two columns:  'alt' and  'lon' 

airports_df.withColumn('altInThousands', 
                       airports_df.alt*airports_df.lon
                      ).show(3)


In [ ]:
# 9. Save the new file with additional column in parquet form

xyz = airports_df.withColumn('altInThousands', airports_df.alt*airports_df.lon)
xyz.write.parquet("hdfs://localhost:9000/user/ashok/data_files/airports_extra.parquet")

In [ ]:
# 9.1 Delete xyz from spark
import gc
del xyz
gc.collect()    # Delete all cache also

In [ ]:
# 9.2 Read the stored parquet file
df = spark.read.parquet("hdfs://localhost:9000/user/ashok/data_files/airports_extra.parquet")
df.show(3)

## Joining tables

In [ ]:
# 9.3 Read 'weather.csv file into spark from hadoop

URL_of_file= "hdfs://localhost:9000/user/ashok/data_files/nycflights/"
weather_df = spark.read.csv(path = URL_of_file + "weather.csv",
                            inferSchema = True,
                            header = True
                           )
weather_df.show(3)

In [ ]:
# 10. Joins
# Refer: http://www.learnbymarketing.com/1100/pyspark-joins-by-example/
# For example, I can join the two titanic dataframes by the column PassengerId

# 10.1
airports_df.join(weather_df, airports_df.faa==weather_df.origin).show(3)
# 10.2
airports_df.join(weather_df, airports_df.faa==weather_df.origin, how = 'inner').show(3)
# 10.3
airports_df.join(weather_df, airports_df.faa==weather_df.origin, how = 'left').show(3)   # Could also use 'left_outer', 'right', 'full'


## SQL queries against DataFrame

In [ ]:
# 11. Many of the operations can be accessed by writing SQL queries in spark.sql().
# To make an existing Spark dataframe usable for spark.sql(), one needs to
#   register said dataframe as a temporary table.

# 11.1 As an example, we can register the two dataframes as temp tables then
#      join them through spark.sql().

airports_df.createOrReplaceTempView('dfa_temp')
weather_df.createOrReplaceTempView('dfw_temp')

In [ ]:
# 11.2 Simple SQL query. SQLContext is no longer needed. 'spark'
#            session object can be used.

dfj = spark.sql('select * from dfa_temp' )
dfj.show(3)

In [ ]:
# 11.3 Now the SQL join

dfj = spark.sql('select * from dfa_temp a, dfw_temp b where a.faa = b.origin' )
dfj.show(3)


## Misc

In [ ]:
# 12. Drop a columns

airports_df.drop('name').show(3)

# 12.1  Or drop multiple columns

columns_to_drop = ['name', 'lat']
xx =airports_df.drop(*columns_to_drop)
xx.show(3)

In [ ]:
########### I am done ####################